In [12]:
import requests, os, shutil

listQuery = ["Emberiza citrinella", "Alauda arvensis", "Emberiza cirlus", "Cuculus canorus", "Tyto alba", "Falco tinnunculus", "Sylvia borin", "Muscicapa striata", "Tachybaptus ruficollis", "Delichon urbicum"]

In [ ]:
url = "https://xeno-canto.org/api/2/recordings?query="

def enregistrer_mp3(url_api, nom_fichier):
    reponse = requests.get(url_api, stream=True)
    
    if reponse.status_code == 200:
        with open(nom_fichier, 'wb') as fichier_audio:
            reponse.raw.decode_content = True
            shutil.copyfileobj(reponse.raw, fichier_audio)
        print("Audio enregistré avec succès sous le nom :", nom_fichier)
    else:
        print("Erreur lors de la récupération des données audio depuis l'API")

def getData(query):
    url_api = url + query
    response = requests.get(url_api)

    if response.status_code == 200:
        contenu = response.json()["recordings"]
        pathToDir = "./data/" + query

        if not os.path.exists(pathToDir):
            os.mkdir(pathToDir)

            for _, element in zip(range(250), contenu):
                elementUrl = element["file"]
                elementFileName = element["file-name"]
                storeFile = pathToDir + "/" + elementFileName
                
                if not os.path.exists(storeFile):
                    enregistrer_mp3(elementUrl, storeFile)

for query in listQuery:
    getData(query)